In [1]:
cd ..

/Users/joeranbosma/stack/Projects/M5Forecast


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, gc
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
import time

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, LeakyReLU
from tensorflow.keras.layers import Flatten, Input, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, Reshape, ReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

from flow import load_data, select_dates, sales_to_money, select_final_day, select_day_nums

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.environ['DATA_DIR'] = 'data/'
os.environ['SUB_DIR'] = 'submissions_uncertainty/'
for dirname, _, filenames in os.walk(os.environ['DATA_DIR']):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

data/.DS_Store
data/empty_pred.csv
data/pred_skeleton.csv
data/calendar.csv
data/sales_train_money.csv
data/sell_prices.csv
data/sales_train_validation.csv
data/sample_submission.csv
data/prep/converted_sales_level_12.csv
data/prep/converted_sales_level_11.csv
data/prep/converted_sales_level_10.csv
data/prep/converted_sales_all.csv
data/prep/norm_level_11.csv
data/prep/.DS_Store
data/prep/norm_level_10.csv
data/prep/norm_level_12.csv
data/prep/norm_level_1.csv
data/prep/norm_level_2.csv
data/prep/norm_level_3.csv
data/prep/norm_level_7.csv
data/prep/norm_level_6.csv
data/prep/converted_sales_level_9.csv
data/prep/norm_level_4.csv
data/prep/norm_level_5.csv
data/prep/converted_sales_level_8.csv
data/prep/converted_sales_level_5.csv
data/prep/norm_level_8.csv
data/prep/norm_level_9.csv
data/prep/converted_sales_level_4.csv
data/prep/converted_sales_level_6.csv
data/prep/converted_sales_level_7.csv
data/prep/converted_sales_level_3.csv
data/prep/norm_all.csv
data/prep/converted_sales_leve

### Load in the data

In [3]:
# Load in the data set
calendar, sales_train_validation, sell_prices = load_data()

sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
sales_train = select_dates(sales_train_validation, day_start=1, num_days=1913-28, include_metadata=True)

### Create a prediction set and a true-sales set

In [4]:
aggregation_levels = {
    1: lambda x: 1,  # global: 1
    2: ['state_id'],  # per state: 3
    3: ['store_id'],  # per store: 10
    4: ['cat_id'],  # per category: 3
    5: ['dept_id'],  # per department: 7
    6: ['state_id', 'cat_id'],  # per state & cat: 9
    7: ['state_id', 'dept_id'],  # per state & dep: 21
    8: ['store_id', 'cat_id'],  # per store & cat: 30
    9: ['store_id', 'dept_id'],  # per store & dep: 70
    10: ['item_id'],  # per item, across stores/states: 3049
    11: ['item_id', 'state_id'],  # per item, across stores: 9,225
    12: ['store_id','item_id']  # lowest level, per product, per store
}

In [5]:
# select true sales period
sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
# Quantiles
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750,0.835, 0.975, 0.995]
columns = ['d_%d' % d for d in range(1886, 1913 + 1)]

data = pd.DataFrame()

np.random.seed(1)
for level, groupby in tqdm(aggregation_levels.items()):
    group = sales_true.groupby(groupby).sum()
            
    group_ = pd.DataFrame()
    for quantile in quantiles:
        g_ = group.copy()
        vals = g_.index.values   
        if level == 1:
            ids = ['Total_X_' + str(quantile)  + '_validation' for val in vals]
        if level in [2,3,4,5,10]:
            ids = [val + '_X_' + str(quantile) + '_validation' for val in vals]
        if level in[6,7,8,9]:
            ids = [val[0] + '_' + val[1] + '_' + str(quantile) + '_validation' for val in vals]
        if level in [11,12]:
            ids = [val[1] + '_' + val[0] + '_' + str(quantile) + '_validation' for val in vals]
            
        for column in g_.columns: 
            if column[:2] == 'd_':
                g_[column] = g_[column]*quantile*np.random.uniform(0.5,1.5)
        
        g_['quantile'] = quantile
        g_['level'] = level
        g_['id'] = ids
        
        group_ = group_.append(g_)
        
    data = data.append(group_)

conv_dict = {'d_1886':'float64','d_1887':'float64','d_1888':'float64','d_1889':'float64','d_1890':'float64','d_1891':'float64',
             'd_1892':'float64','d_1893':'float64','d_1894':'float64','d_1895':'float64','d_1896':'float64','d_1897':'float64',
             'd_1898':'float64','d_1899':'float64','d_1900':'float64','d_1901':'float64','d_1902':'float64','d_1903':'float64',
             'd_1904':'float64','d_1905':'float64','d_1906':'float64','d_1907':'float64','d_1908':'float64','d_1909':'float64',
             'd_1910':'float64','d_1911':'float64','d_1912':'float64','d_1913':'float64'}

data = data.astype(conv_dict)
data.columns = ['F%d' % int(i+1) if x =='d_1' + str(i+886) else x for i, x in enumerate(data.columns)]

data.head()

100%|██████████| 12/12 [00:03<00:00,  3.72it/s]


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F22,F23,F24,F25,F26,F27,F28,quantile,level,id
1,165.251950,206.582632,80.916005,139.128480,147.253381,155.998292,193.611162,170.879367,177.932118,203.265270,...,279.402837,152.801732,210.701291,241.095206,282.526391,143.224673,134.211165,0.005,1,Total_X_0.005_validation
1,603.533954,1166.494268,484.047630,798.623339,1659.661441,1360.472047,1681.291654,824.037769,1177.097908,1305.731026,...,599.456436,487.815415,1041.589606,623.261695,775.441350,1213.735126,688.867198,0.025,1,Total_X_0.025_validation
1,6387.524982,3612.887744,5816.058248,6865.455248,4525.603591,7943.950903,11120.014768,6096.634123,3600.927513,6688.897042,...,5323.744941,7753.846768,7149.523721,7996.124778,7512.091424,10106.026170,6974.697335,0.165,1,Total_X_0.165_validation
1,6937.242787,11815.129922,7508.025703,12700.529419,13244.618011,14770.489244,8671.760381,14646.364215,9423.840798,10550.424792,...,10688.490662,4844.851023,12630.150038,10430.172928,15166.757483,8229.783992,7931.543663,0.250,1,Total_X_0.25_validation
1,25816.086542,20260.336247,9157.599795,21770.357064,28548.253059,37476.774168,34180.142253,12615.891870,10315.201676,10296.370380,...,9869.598593,24432.384756,12952.754880,22895.908159,17986.724858,33380.868141,31050.211100,0.500,1,Total_X_0.5_validation


In [6]:
# select true sales period
sales_true = select_dates(sales_train_validation, day_end=1913, num_days=28, include_metadata=True)
# Quantiles
quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750,0.835, 0.975, 0.995]
columns = ['d_%d' % d for d in range(1886, 1913 + 1)]

true_sales = pd.DataFrame()
for level, groupby in tqdm(aggregation_levels.items()):
    group = sales_true.groupby(groupby).sum()
            
    group_ = pd.DataFrame()
    for quantile in quantiles:
        g_ = group.copy()
        vals = g_.index.values   
        if level == 1:
            ids = ['Total_X_' + str(quantile)  + '_validation' for val in vals]
        if level in [2,3,4,5,10]:
            ids = [val + '_X_' + str(quantile) + '_validation' for val in vals]
        if level in[6,7,8,9]:
            ids = [val[0] + '_' + val[1] + '_' + str(quantile) + '_validation' for val in vals]
        if level in [11,12]:
            ids = [val[1] + '_' + val[0] + '_' + str(quantile) + '_validation' for val in vals]
        
        g_['quantile'] = quantile
        g_['level'] = str(level)
        g_['id'] = ids
        
        group_ = group_.append(g_)
        
    true_sales = true_sales.append(group_)

conv_dict = {'d_1886':'float64','d_1887':'float64','d_1888':'float64','d_1889':'float64','d_1890':'float64','d_1891':'float64',
             'd_1892':'float64','d_1893':'float64','d_1894':'float64','d_1895':'float64','d_1896':'float64','d_1897':'float64',
             'd_1898':'float64','d_1899':'float64','d_1900':'float64','d_1901':'float64','d_1902':'float64','d_1903':'float64',
             'd_1904':'float64','d_1905':'float64','d_1906':'float64','d_1907':'float64','d_1908':'float64','d_1909':'float64',
             'd_1910':'float64','d_1911':'float64','d_1912':'float64','d_1913':'float64'}

true_sales = true_sales.astype(conv_dict)
true_sales.columns = ['F%d' % int(i+1) if x =='d_1' + str(i+886) else x for i, x in enumerate(true_sales.columns)]

true_sales.head()

100%|██████████| 12/12 [00:01<00:00,  6.82it/s]


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F22,F23,F24,F25,F26,F27,F28,quantile,level,id
1,36041.0,33857.0,32359.0,34681.0,45536.0,52672.0,56425.0,40418.0,39683.0,39134.0,...,38059.0,37570.0,35343.0,35033.0,40517.0,48962.0,49795.0,0.005,1,Total_X_0.005_validation
1,36041.0,33857.0,32359.0,34681.0,45536.0,52672.0,56425.0,40418.0,39683.0,39134.0,...,38059.0,37570.0,35343.0,35033.0,40517.0,48962.0,49795.0,0.025,1,Total_X_0.025_validation
1,36041.0,33857.0,32359.0,34681.0,45536.0,52672.0,56425.0,40418.0,39683.0,39134.0,...,38059.0,37570.0,35343.0,35033.0,40517.0,48962.0,49795.0,0.165,1,Total_X_0.165_validation
1,36041.0,33857.0,32359.0,34681.0,45536.0,52672.0,56425.0,40418.0,39683.0,39134.0,...,38059.0,37570.0,35343.0,35033.0,40517.0,48962.0,49795.0,0.250,1,Total_X_0.25_validation
1,36041.0,33857.0,32359.0,34681.0,45536.0,52672.0,56425.0,40418.0,39683.0,39134.0,...,38059.0,37570.0,35343.0,35033.0,40517.0,48962.0,49795.0,0.500,1,Total_X_0.5_validation


In [7]:
def convert_true_sales_to_quantiles(sales_true, aggregation_levels):
    # Quantiles
    quantiles = [0.005, 0.025, 0.165, 0.250, 0.500, 0.750,0.835, 0.975, 0.995]
    d_cols = select_day_nums(sales_true, as_int=False)

    sales_true_quantiles = pd.DataFrame()
    for level, groupby in tqdm(aggregation_levels.items()):
        if level == 12:
            groupby = ['store_id', 'item_id']
        group = sales_true.groupby(groupby).sum()

        group_ = pd.DataFrame()
        for quantile in quantiles:
            g_ = group.copy()
            vals = g_.index.values
            if level == 1:
                ids = ['Total_X_' + str(quantile)  + '_validation' for val in vals]
            if level in [2, 3, 4, 5, 10]:
                ids = [val + '_X_' + str(quantile) + '_validation' for val in vals]
            if level in[6, 7, 8, 9]:
                ids = [val[0] + '_' + val[1] + '_' + str(quantile) + '_validation' for val in vals]
            if level in [11, 12]:
                ids = [val[1] + '_' + val[0] + '_' + str(quantile) + '_validation' for val in vals]

            g_['quantile'] = quantile
            g_['level'] = level
            g_['id'] = ids

            group_ = group_.append(g_)

        sales_true_quantiles = sales_true_quantiles.append(group_)

    # convert quantiles to float
    conv_dict = {d_col: 'float64' for d_col in d_cols}
    sales_true_quantiles = sales_true_quantiles.astype(conv_dict)
    sales_true_quantiles.columns = ['F%d' % int(i+1) if x =='d_1' + str(i+886) else x
                                    for i, x in enumerate(sales_true_quantiles.columns)]

    return sales_true_quantiles

sales_true_quantiles = convert_true_sales_to_quantiles(sales_true, aggregation_levels)

100%|██████████| 12/12 [00:02<00:00,  4.72it/s]


In [8]:
true_sales.iloc[:, 0:28].equals( sales_true_quantiles.iloc[:, 0:28] )

True

### New referee for SPL given predictions and true quantile sales

In [9]:
def day_to_int(day, default=None):
    try:
        return int(day[2:])
    except (ValueError, TypeError) as e:
        return default

class RefereeTest(object):
    def __init__(self, sales_true, sales_train, prices, calendar, verbose=True):
        if verbose: print("Initializing Referee")
        self.sales_true = sales_true
        self.sales_train = sales_train
        self.h = sales_true.shape[1]
        self.n = sales_train.shape[1]
        
        self.quantiles = np.array([0.005, 0.025, 0.165, 0.250, 0.500, 0.750, 0.835, 0.975, 0.995])

        # Define aggregation levels as their Pandas groupby
        # Follow same order as https://github.com/Mcompetitions/M5-methods/blob/master/validation/Point%20Forecasts%20-%20Benchmarks.R
        self.aggregation_levels = {
            1: lambda x: 1,  # global: 1
            2: ['state_id'],  # per state: 3
            3: ['store_id'],  # per store: 10
            4: ['cat_id'],  # per category: 3
            5: ['dept_id'],  # per department: 7
            6: ['state_id', 'cat_id'],  # per state & cat: 9
            7: ['state_id', 'dept_id'],  # per state & dep: 21
            8: ['store_id', 'cat_id'],  # per store & cat: 30
            9: ['store_id', 'dept_id'],  # per store & dep: 70
            10: ['item_id'],  # per item, across stores/states: 3049
            11: ['item_id', 'state_id'],  # per item, across stores: 9,225
            12: ['store_id', 'item_id']  # lowest level, per product, per store
        }
        
        self.sales_true_quantiles = convert_true_sales_to_quantiles(sales_true, self.aggregation_levels)

        # Set number of aggregation levels
        self.K = len(self.aggregation_levels)  # 12 for full evaluation

        # Calculate weights (based on cumulative dollars per product, in last 28 days of train data)
        # Select final 28 days of training data
        last_train_day = select_final_day(sales_train)
        sales_train_final = select_dates(sales_train, day_end=last_train_day, num_days=28, include_metadata=True)

        # Try to find money spent pre-converted
        fn = os.environ['DATA_DIR'] + "/sales_train_money.csv"
        if os.path.exists(fn):
            sales_train_money = pd.read_csv(fn, index_col='id')
            sales_train_final = select_dates(sales_train_money, day_end=last_train_day, num_days=28, include_metadata=True)
        else:
            # Convert quantities sold to money spent
            sales_train_final = sales_to_money(sales_train_final, prices, calendar, verbose=verbose)

        # Calculate weights of each level
        if verbose: print("Calculating weights for each level...")
        self.weights = {}
        for level, groupby in self.aggregation_levels.items():
            self.weights[level] = self.calc_weights(sales_train_final, groupby=groupby)

        # Calculate scale of each level
        if verbose: print("Calculating scale for each level...")
        self.scales = {}
        for level, groupby in self.aggregation_levels.items():
            self.scales[level] = self.calc_scale(groupby=groupby)

        if verbose: print("Finished setup.")
            
    def get_scales(self):
        return self.scales
    
    def get_weights(self):
        return self.weights

    def calc_weights(self, sales, groupby=None):
        """From the docs: Since the M5 competition involves twelve aggregation levels, K is set equal to 12,
        with the weights of the series being computed so that they sum to one at each aggregation level."""
        # Calculate aggregate dollars spent
        if groupby:
            sales = sales.groupby(groupby).sum()

        weights = sales.sum(axis=1)

        # Normalize weights to sum to one
        weights /= weights.sum()
        return weights

    def calc_scale(self, groupby=None):
        # Calculate scale: 1/(n-1) * sum |Yt - Y_t-1|
        """As done with RMSSE, the denominator of SPL is computed only for the time-periods for which the examined
        items/products are actively sold, i.e., the periods following the first non-zero demand observed for the
        series under evaluation. 
        """
        if groupby:
            Yt = self.sales_train.groupby(groupby).sum()
            Yt1 =Yt.shift(1, axis=1)
        else:
            day_cols = self.sales_train.filter(regex='d_').columns
            Yt = self.sales_train[day_cols]
            Yt1 = Yt.shift(1, axis=1)
        
        # Calculate number of sales since fist sale for each product (default is required when unit is not sold in
        # training period, which also makes its weight zero.)
        first_sold_day = Yt.replace(0, np.nan).apply(lambda x: day_to_int(x.first_valid_index(), default=-1), axis=1)
        n = select_final_day(Yt) - first_sold_day  # list of numbers since first sale

        scale = (np.abs(Yt - Yt1).sum(axis=1) / (n - 1))
        return scale
    
 
    def evaluate_SPL(self, quantiles_pred):
        """Evaluate the Scaled Pinball Loss for a given set of predictions"""
        metrics = {}

        # Determine predicted levels
        predicted_levels = quantiles_pred.level.unique()

        # Calculate SPL for each level
        for level, groupby in self.aggregation_levels.items():
            if level in predicted_levels:
                # The groupby, weights and scale will be selected using the level
                metrics[level] = self.calc_SPL(quantiles_pred, level=level, groupby=groupby)

        SPL = np.mean(list(metrics.values()))  # or sum and divide by self.K, take average over all aggregation levels
        metrics['WSPL'] = SPL
        return metrics

    def calc_SPL(self, quantiles_pred, level=None, groupby=None, weights=None, scale=None):
        """Calculate the Scaled Pinball Loss for a given aggregation level"""
        if level:
            if groupby is None: groupby = self.aggregation_levels[level]
            if weights is None: weights = self.weights[level]
            if scale is None: scale = self.scales[level]
        assert weights is not None, "Provide level or weights"
        assert scale is not None, "Provide level or scale"

        # Select the correct predictions and true sales based on the input level
        predictions = quantiles_pred[quantiles_pred['level'] == level]
        true_sales = self.sales_true_quantiles[self.sales_true_quantiles['level'] == level]

        # Make sure that both the predictions and the true sales have the same
        # id list, otherwise our calculation will go wrong
        predictions = predictions.sort_values('id')
        true_sales = true_sales.sort_values('id')

        # Convert to numpy array
        predictions = predictions.to_numpy()[:,1:29]
        true_sales = true_sales.to_numpy()[:,1:29]

        # Error
        err = true_sales - predictions

        # Number of rows
        Nlevel = predictions.shape[0]

        # Dummy array to save losses in
        losses = np.zeros(Nlevel // 9)
        for i in range(Nlevel // 9):
            indices = np.arange(i*9,(i+1)*9) # per set of 9, take indices
            subset = err[indices] # Take subset out of real set
            res = np.mean(np.sum(np.amax(np.array([self.quantiles * subset.T, (self.quantiles - 1) * subset.T]),axis=0),axis=0)) #compute PL of set
            losses[i] = res # Save resulting PL

        loss = np.sum(np.array(losses * weights)/np.array(self.h*scale)) # Calculate SPL of aggregate level

        return loss

### Test our created (random) prediction set

In [10]:
ref_test = RefereeTest(sales_true, sales_train, sell_prices, calendar, verbose=True)

ref_test.evaluate_SPL(data)

 25%|██▌       | 3/12 [00:00<00:00, 29.20it/s]

Initializing Referee


100%|██████████| 12/12 [00:02<00:00,  4.36it/s]


Calculating weights for each level...
Calculating scale for each level...
Finished setup.


{1: 1.1210214349050245,
 2: 1.0153875855884373,
 3: 0.9212650321985366,
 4: 1.1225041859572058,
 5: 0.9974407717372311,
 6: 0.9666953992823991,
 7: 0.8645371305215135,
 8: 0.8413401753512274,
 9: 0.822862237419191,
 10: 0.4316011238689629,
 11: 0.23945040291756978,
 12: 0.1481145269528085,
 'WSPL': 0.7910183338916755}

In [11]:
# only evaluate certain levels
ref_test.evaluate_SPL(data[(data.level == 12) | (data.level == 3)])

{3: 0.9212650321985366, 12: 0.1481145269528085, 'WSPL': 0.5346897795756725}

### Move to evaluation.py

In [12]:
from evaluation import Referee

In [13]:
ref = Referee(sales_true, sales_train, sell_prices, calendar, verbose=True)

Initializing Referee
Converting true sales to quantile form
Calculating weights for each level...
Calculating scale for each level...
Finished setup.


In [14]:
ref.evaluate_SPL(data)

{1: 1.1210214349050245,
 2: 1.0153875855884373,
 3: 0.9212650321985366,
 4: 1.1225041859572058,
 5: 0.9974407717372311,
 6: 0.9666953992823991,
 7: 0.8645371305215135,
 8: 0.8413401753512274,
 9: 0.822862237419191,
 10: 0.4316011238689629,
 11: 0.23945040291756978,
 12: 0.1481145269528085,
 'WSPL': 0.7910183338916755}

In [15]:
# only evaluate certain levels
ref.evaluate_SPL(data[(data.level == 12) | (data.level == 3)])

{3: 0.9212650321985366, 12: 0.1481145269528085, 'WSPL': 0.5346897795756725}